In [2]:
import pyspark
import os
import pyspark
import os
del os.environ['PYSPARK_SUBMIT_ARGS']
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.types import *
import pyspark.sql.functions as F
#spark = SparkSession.builder.appName('Graphs-HW2').getOrCreate()

In [3]:
from pyspark.sql.functions import lit

In [4]:
spark = SparkSession.builder.appName('Graphs-HW2').getOrCreate()

In [5]:
# Read lines from the text file
answers_sdf = spark.read.load('sx-stackoverflow-a2q.txt', format="text")
comments_answers_sdf = spark.read.load('sx-stackoverflow-c2a.txt', format="text")
comments_questions_sdf = spark.read.load('sx-stackoverflow-c2q.txt', format="text")

## Step 2.2

In [7]:
answers_sdf.count()

17823525

In [8]:
answers_sdf.show(10)

+----------------+
|           value|
+----------------+
|  9 8 1217567877|
|  1 1 1217573801|
| 13 1 1217606247|
| 17 1 1217617639|
| 48 2 1217618182|
| 17 1 1217618239|
| 19 9 1217618357|
|13 23 1217618560|
|13 11 1217618799|
|23 23 1217619360|
+----------------+
only showing top 10 rows



In [9]:
answers_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [10]:
comments_answers_sdf.count() 

25405374

In [11]:
comments_answers_sdf.show(10)

+--------------------+
|               value|
+--------------------+
|     1 91 1220713630|
|     3 91 1220713792|
|  380 350 1220715736|
|4642 2257 1220734307|
|4642 1324220 1220...|
|2495 4285 1220736640|
|4642 4893 1220737355|
|2515 4903 1220738560|
|2515 4893 1220739071|
|  199 199 1220741079|
+--------------------+
only showing top 10 rows



In [14]:
comments_answers_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [12]:
comments_questions_sdf.count()

20268151

In [13]:
comments_questions_sdf.show(10)

+--------------------+
|               value|
+--------------------+
|4550 4550 1220729190|
|  242 184 1220733503|
|4213 4946 1220768149|
|    91 91 1220768295|
|2658 1874 1220771891|
|4035 1874 1220773037|
|2257 4489 1220802041|
|  577 577 1220834891|
|4489 4489 1220853536|
| 828 2783 1220854143|
+--------------------+
only showing top 10 rows



In [14]:
comments_questions_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [7]:
comments_questions_sdf.createOrReplaceTempView('comments_questions')

In [8]:
comments_questions_sdf = spark.sql('select CAST(split(value, " ")[0] AS integer) AS from_node,'\
                     'CAST(split(value, " ")[1]  AS integer) AS to_node from comments_questions')

In [9]:
comments_questions_sdf.show(10)

+---------+-------+
|from_node|to_node|
+---------+-------+
|     4550|   4550|
|      242|    184|
|     4213|   4946|
|       91|     91|
|     2658|   1874|
|     4035|   1874|
|     2257|   4489|
|      577|    577|
|     4489|   4489|
|      828|   2783|
+---------+-------+
only showing top 10 rows



In [21]:
comments_questions_sdf.createOrReplaceTempView('comments_questions')

In [22]:
comments_questions_sdf = comments_questions_sdf.withColumn('comment-on-question', F.lit('value'))

In [23]:
answers_sdf.createOrReplaceTempView('answers')

In [24]:
answers_sdf = spark.sql('select CAST(split(value, " ")[0] AS integer) AS from_node,'\
                     'CAST(split(value, " ")[1]  AS integer) AS to_node, from_unixtime(split(value, " ")[2]) AS timestamp from answers')

In [25]:
answers_sdf = answers_sdf.withColumn('answer', F.lit('value'))

In [26]:
answers_sdf.show(10)

+---------+-------+-------------------+------+
|from_node|to_node|          timestamp|answer|
+---------+-------+-------------------+------+
|        9|      8|2008-08-01 01:17:57| value|
|        1|      1|2008-08-01 02:56:41| value|
|       13|      1|2008-08-01 11:57:27| value|
|       17|      1|2008-08-01 15:07:19| value|
|       48|      2|2008-08-01 15:16:22| value|
|       17|      1|2008-08-01 15:17:19| value|
|       19|      9|2008-08-01 15:19:17| value|
|       13|     23|2008-08-01 15:22:40| value|
|       13|     11|2008-08-01 15:26:39| value|
|       23|     23|2008-08-01 15:36:00| value|
+---------+-------+-------------------+------+
only showing top 10 rows



In [27]:
answers_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- answer: string (nullable = false)



In [28]:
comments_answers_sdf.createOrReplaceTempView('comments_answers')

In [29]:
comments_answers_sdf = spark.sql('select CAST(split(value, " ")[0] AS integer) AS from_node,'\
                     'CAST(split(value, " ")[1]  AS integer) AS to_node, from_unixtime(split(value, " ")[2]) AS timestamp from comments_answers')

In [30]:
comments_answers_sdf = comments_answers_sdf.withColumn('comment-on-answer', F.lit('value'))

In [31]:
comments_answers_sdf.show(10)

+---------+-------+-------------------+-----------------+
|from_node|to_node|          timestamp|comment-on-answer|
+---------+-------+-------------------+-----------------+
|        1|     91|2008-09-06 11:07:10|            value|
|        3|     91|2008-09-06 11:09:52|            value|
|      380|    350|2008-09-06 11:42:16|            value|
|     4642|   2257|2008-09-06 16:51:47|            value|
|     4642|1324220|2008-09-06 17:15:46|            value|
|     2495|   4285|2008-09-06 17:30:40|            value|
|     4642|   4893|2008-09-06 17:42:35|            value|
|     2515|   4903|2008-09-06 18:02:40|            value|
|     2515|   4893|2008-09-06 18:11:11|            value|
|      199|    199|2008-09-06 18:44:39|            value|
+---------+-------+-------------------+-----------------+
only showing top 10 rows



In [32]:
graph_sdf = answers_sdf.unionAll(comments_questions_sdf).unionAll(comments_answers_sdf)

## Step 2.3

In [33]:
answers_sdf.count()

17823525

In [34]:
answers_sdf.show(5)

+---------+-------+-------------------+------+
|from_node|to_node|          timestamp|answer|
+---------+-------+-------------------+------+
|        9|      8|2008-08-01 01:17:57| value|
|        1|      1|2008-08-01 02:56:41| value|
|       13|      1|2008-08-01 11:57:27| value|
|       17|      1|2008-08-01 15:07:19| value|
|       48|      2|2008-08-01 15:16:22| value|
+---------+-------+-------------------+------+
only showing top 5 rows



In [35]:
answers_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- answer: string (nullable = false)



In [36]:
comments_answers_sdf.count()

25405374

In [37]:
comments_answers_sdf.show(5)

+---------+-------+-------------------+-----------------+
|from_node|to_node|          timestamp|comment-on-answer|
+---------+-------+-------------------+-----------------+
|        1|     91|2008-09-06 11:07:10|            value|
|        3|     91|2008-09-06 11:09:52|            value|
|      380|    350|2008-09-06 11:42:16|            value|
|     4642|   2257|2008-09-06 16:51:47|            value|
|     4642|1324220|2008-09-06 17:15:46|            value|
+---------+-------+-------------------+-----------------+
only showing top 5 rows



In [38]:
comments_answers_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- comment-on-answer: string (nullable = false)



In [39]:
comments_questions_sdf.count()

20268151

In [40]:
comments_questions_sdf.show(5)

+---------+-------+-------------------+-------------------+
|from_node|to_node|          timestamp|comment-on-question|
+---------+-------+-------------------+-------------------+
|     4550|   4550|2008-09-06 15:26:30|              value|
|      242|    184|2008-09-06 16:38:23|              value|
|     4213|   4946|2008-09-07 02:15:49|              value|
|       91|     91|2008-09-07 02:18:15|              value|
|     2658|   1874|2008-09-07 03:18:11|              value|
+---------+-------+-------------------+-------------------+
only showing top 5 rows



In [41]:
comments_questions_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- comment-on-question: string (nullable = false)



In [42]:
graph_sdf.count()

63497050

In [43]:
graph_sdf.show(5)


+---------+-------+-------------------+------+
|from_node|to_node|          timestamp|answer|
+---------+-------+-------------------+------+
|        9|      8|2008-08-01 01:17:57| value|
|        1|      1|2008-08-01 02:56:41| value|
|       13|      1|2008-08-01 11:57:27| value|
|       17|      1|2008-08-01 15:07:19| value|
|       48|      2|2008-08-01 15:16:22| value|
+---------+-------+-------------------+------+
only showing top 5 rows



In [44]:
graph_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- answer: string (nullable = false)



In [45]:
outdegree_sdf = graph_sdf.groupby(graph_sdf['from_node']).count()

In [46]:
outdegree_sdf.show(20)

+---------+-----+
|from_node|count|
+---------+-----+
|      463|   77|
|     1088|  939|
|     1238|   11|
|     1645|   11|
|     1580|   94|
|     1959|  137|
|     2659|   22|
|     3175|   72|
|     2122|  122|
|     3997|   28|
|     4101|    2|
|     3749|    1|
|     4900|   76|
|     5156|   32|
|      148|  100|
|     6336|   61|
|     7754|   85|
|     8086|   17|
|     6466|   22|
|     9900|   66|
+---------+-----+
only showing top 20 rows



In [47]:
outdegree_sdf.createOrReplaceTempView('outdegree')

In [48]:
outdegree_sdf = spark.sql('SELECT from_node, count FROM outdegree ORDER BY count desc')

In [49]:
indegree_sdf = graph_sdf.groupby(graph_sdf['to_node']).count()

In [50]:
indegree_sdf.createOrReplaceTempView('indegree')

In [51]:
indegree_sdf = spark.sql('SELECT to_node, count FROM indegree ORDER BY count desc')

## Step 3

In [52]:
indegree_sdf.show(5)

+-------+-----+
|to_node|count|
+-------+-----+
|  22656|93143|
| 548225|54410|
|  29407|45824|
| 157882|39576|
|1144035|38909|
+-------+-----+
only showing top 5 rows



In [53]:
outdegree_sdf.show(5)

+---------+------+
|from_node| count|
+---------+------+
|    22656|101663|
|   505088| 65522|
|    17034| 65260|
|  1491895| 62746|
|   100297| 54330|
+---------+------+
only showing top 5 rows



In [54]:
comments_answers_sdf.createOrReplaceTempView('comments_answers')

In [55]:
def spark_bfs(g_sdf,origin,depth):
        schema = StructType([
             StructField("node", IntegerType(), True)
        ])
        schema2 = StructType([
             StructField("node", IntegerType(), True),
             StructField("depth", IntegerType(), True)
        ])
        g_sdf.createOrReplaceTempView('g_sdf')
        finaldepths_sdf = spark.createDataFrame([],schema2)
        for item in origin:
             value = int(item["node"])
             #set visit_sdf to origin node
             visit_sdf = spark.createDataFrame([item],schema)
             #set frontier_sdf to to_node from origin (exclude pointing to self)
             g2_sdf = g_sdf[g_sdf.from_node==value]
             g2_sdf = g2_sdf[g_sdf.to_node!=value]
             g2_sdf.createOrReplaceTempView('g2_sdf')
             frontier_sdf = spark.sql('SELECT to_node as node FROM g2_sdf')
             frontier_sdf.createOrReplaceTempView('frontier_sdf')
             visit_sdf.createOrReplaceTempView('visit_sdf')
             depths_sdf = spark.createDataFrame([{'node':item['node'],'depth':0}],schema2)
             #iterative case
             frontier_sdf.repartition(100, frontier_sdf.node)
             for i in range(1,depth+1): 
                  #find distinct nodes from frontier (want distinct to prevent duplicate propagation)
                  #for finding to_nodes (placed in results)
                  result_sdf = spark.sql('SELECT DISTINCT node FROM frontier_sdf')
                  result_sdf.createOrReplaceTempView('result_sdf')
                  result_sdf = spark.sql('SELECT * FROM result_sdf f '\
                           'LEFT JOIN g_sdf g ON f.node == g.from_node').cache()
                  result_sdf.createOrReplaceTempView('result_sdf')
                  #remove those already visited
                  result_sdf = spark.sql('SELECT * FROM result_sdf r '\
                           'LEFT JOIN visit_sdf v ON r.to_node == v.node '\
                           'WHERE v.node IS NULL').cache()
                  #remove reflexive property (to_node == from_node)
                  #this prevents a node from having multiple depths
                  result_sdf = result_sdf[result_sdf.from_node != result_sdf.to_node]
                  result_sdf.createOrReplaceTempView('result_sdf')
                  result_sdf = result_sdf.filter("to_node is not null").cache()
                  #update visit to add current frontier
                  visit_sdf = visit_sdf.unionAll(frontier_sdf).cache()
                  #update depths to add current frontier
                  depths_sdf = depths_sdf.unionAll(frontier_sdf.withColumn('depth', F.lit(i)))
                  result_sdf.createOrReplaceTempView('result_sdf')
                  #update frontier to to_node results, renamed node
                  frontier_sdf = spark.sql('SELECT to_node as node from result_sdf').cache()
                  frontier_sdf.createOrReplaceTempView('frontier_sdf')
                  frontier_sdf.repartition(100, frontier_sdf.node)
                  visit_sdf.repartition(100, visit_sdf.node)
             finaldepths_sdf = finaldepths_sdf.unionAll(depths_sdf)
                
        return finaldepths_sdf

In [10]:
gt_list = [{'from_node':1,'to_node':2},{'from_node':6,'to_node':8},\
           {'from_node':1,'to_node':3},{'from_node':1,'to_node':4},\
           {'from_node':1,'to_node':5},{'from_node':1,'to_node':6},\
           {'from_node':1,'to_node':7},{'from_node':7,'to_node':9},\
           {'from_node':7,'to_node':7},{'from_node':8,'to_node':8},\
           {'from_node':7,'to_node':13},{'from_node':7,'to_node':14},\
           {'from_node':2,'to_node':9},{'from_node':5,'to_node':8},\
           {'from_node':6,'to_node':1},{'from_node':2,'to_node':10},\
           {'from_node':4,'to_node':10},{'from_node':10,'to_node':11},\
           {'from_node':11,'to_node':12},{'from_node':9,'to_node':1},\
           {'from_node':1,'to_node':1}]

In [11]:
schema3 = StructType([
             StructField("from_node", IntegerType(),True),
             StructField("to_node", IntegerType(), True)
        ])
gt_sdf = spark.createDataFrame(gt_list,schema3)

In [12]:
gt_sdf.show()

+---------+-------+
|from_node|to_node|
+---------+-------+
|        1|      2|
|        6|      8|
|        1|      3|
|        1|      4|
|        1|      5|
|        1|      6|
|        1|      7|
|        7|      9|
|        7|      7|
|        8|      8|
|        7|     13|
|        7|     14|
|        2|      9|
|        5|      8|
|        6|      1|
|        2|     10|
|        4|     10|
|       10|     11|
|       11|     12|
|        9|      1|
+---------+-------+
only showing top 20 rows



In [59]:
origins2 = [{'node':4550},{'node': 242}]

## Step 4.1

In [60]:
depth_sdf = spark_bfs(comments_questions_sdf,origins2,2)
#depth_sdf.show()

In [61]:
depth_sdf.show() #answer for graph traversal

+-------+-----+
|   node|depth|
+-------+-----+
|   4550|    0|
|    242|    0|
|    184|    1|
|  45459|    1|
|   1551|    1|
|  67719|    1|
| 451383|    1|
|1139389|    1|
| 633513|    1|
| 602291|    1|
|1619254|    1|
| 171136|    1|
|1940564|    1|
|2605058|    1|
|2705465|    1|
|2495819|    1|
| 818089|    1|
|1684384|    1|
|3047450|    1|
|2332659|    1|
+-------+-----+
only showing top 20 rows



In [62]:
depth_sdf.count()  #46764

397

In [55]:
test_sdf = spark_bfs(gt_sdf,[{'node':1}],4) #toy model test

NameError: name 'spark_bfs' is not defined

In [65]:
test_sdf.show()

+----+-----+
|node|depth|
+----+-----+
|   1|    0|
|   2|    1|
|   3|    1|
|   4|    1|
|   5|    1|
|   6|    1|
|   7|    1|
|  13|    2|
|   9|    2|
|   9|    2|
|   8|    2|
|   8|    2|
|  10|    2|
|  10|    2|
|  14|    2|
|  11|    3|
|  12|    4|
+----+-----+



In [66]:
origin = test_sdf[test_sdf.depth == 0]
origin.show()

+----+-----+
|node|depth|
+----+-----+
|   1|    0|
+----+-----+



In [67]:
def friend_rec(filtered_bfs_sdf,graph_sdf):
    mod_sdf = filtered_bfs_sdf[filtered_bfs_sdf.depth == 2]
    origin_sdf = filtered_bfs_sdf[filtered_bfs_sdf.depth == 0]
    mod = mod_sdf.groupBy('node').count().collect()
    schema4 = StructType([
             StructField("node", IntegerType(), True),
             StructField("count", IntegerType(), True)
        ])
    mod_sdf = spark.createDataFrame(mod,schema4)
    mod_sdf.createOrReplaceTempView('mod_sdf')
    #collect only nodes that are at depth 2 and have duplicates
    mod_sdf = spark.sql('SELECT node from mod_sdf WHERE count > 1')
    mod_sdf.createOrReplaceTempView('mod_sdf')
    graph_sdf.createOrReplaceTempView('graph_sdf')
    #find the to_node from the original graph from nodes of depth 2
    result_sdf = spark.sql('SELECT * FROM mod_sdf f '\
                           'LEFT JOIN graph_sdf g ON f.node == g.from_node').cache()
    result_sdf = result_sdf.withColumnRenamed('node', 'possible_friends')
    origin_sdf.createOrReplaceTempView('origin_sdf')
    result_sdf.createOrReplaceTempView('result_sdf')
    #remove nodes that are not connected back to origin
    result_sdf = spark.sql('SELECT * FROM result_sdf r '\
                           'LEFT JOIN origin_sdf o ON r.to_node == o.node '\
                           'WHERE o.node IS NULL').cache()
    #result_sdf = result_sdf.filter("to_node is not null").cache()
    result_sdf.createOrReplaceTempView('result_sdf')
    friends_sdf = spark.sql('SELECT DISTINCT possible_friends as node_2 from result_sdf').cache()
    friends_sdf = friends_sdf.withColumn('node_1', F.lit("242 or 4550"))
    #final friends list should be: depth 2, duplicates, no link to origin
    
    return friends_sdf

In [68]:
friends_sdf = friend_rec(test_sdf,gt_sdf)

In [69]:
friends_sdf = friend_rec(depth_sdf,comments_questions_sdf)

In [70]:
depth_sdf.createOrReplaceTempView('depth_sdf')

## Step 4.2

In [71]:
filtered_bfs_sdf = depth_sdf[depth_sdf.depth==2]

In [72]:
friends_sdf = friend_rec(depth_sdf,comments_questions_sdf) #filtering occurs within function

In [77]:
friends_sdf.count()

42

In [73]:
friends_sdf.show()

+-------+-----------+
| node_2|     node_1|
+-------+-----------+
| 104015|242 or 4550|
| 713813|242 or 4550|
|4800193|242 or 4550|
|  55503|242 or 4550|
| 238419|242 or 4550|
| 102427|242 or 4550|
|  27483|242 or 4550|
|2269511|242 or 4550|
|4204628|242 or 4550|
|4595831|242 or 4550|
|  60602|242 or 4550|
|  63775|242 or 4550|
|  24872|242 or 4550|
| 139605|242 or 4550|
|  47550|242 or 4550|
| 132640|242 or 4550|
|1406214|242 or 4550|
| 318811|242 or 4550|
|3565829|242 or 4550|
| 282194|242 or 4550|
+-------+-----------+
only showing top 20 rows



In [16]:
def transitive_closure(g_sdf,origins_sdf,depth):
        schema2 = StructType([
             StructField("node", IntegerType(), True),
        ])
        g_sdf.createOrReplaceTempView('g_sdf')
        origins_sdf.createOrReplaceTempView('origins_sdf')
        finaldepths_sdf = spark.createDataFrame([],schema2)
        #move fom_node in origins list to visited
        visit_sdf = spark.sql('SELECT from_node as vnode FROM origins_sdf')
        visit_sdf = visit_sdf.distinct()
        frontier_sdf = spark.sql('SELECT to_node as node from origins_sdf').cache()
        frontier_sdf.createOrReplaceTempView('frontier_sdf')
        visit_sdf.createOrReplaceTempView('visit_sdf')
        #remove from frontier nodes that are already visited
        frontier_sdf = spark.sql('SELECT * FROM frontier_sdf f '\
                           'LEFT JOIN visit_sdf v ON f.node == v.vnode '\
                           'WHERE v.vnode IS NULL').cache()
        frontier_sdf.createOrReplaceTempView('frontier_sdf')
        frontier_sdf = spark.sql('SELECT node from frontier_sdf')
        for i in range(0,depth):
             frontier_sdf.createOrReplaceTempView('frontier_sdf')
             visit_sdf.createOrReplaceTempView('visit_sdf')
             result_sdf = spark.sql('SELECT DISTINCT node FROM frontier_sdf')
             result_sdf.createOrReplaceTempView('result_sdf')
             result_sdf = spark.sql('SELECT * FROM result_sdf r '\
                           'LEFT JOIN g_sdf g ON r.node == g.from_node').cache()
             result_sdf.createOrReplaceTempView('result_sdf')
             #remove those already visited
             result_sdf = spark.sql('SELECT * FROM result_sdf r '\
                           'LEFT JOIN visit_sdf v ON r.to_node == v.vnode '\
                           'WHERE v.vnode IS NULL').cache()
             #remove reflexive property (to_node == from_node)
             #this prevents a node from having multiple depths
             result_sdf.createOrReplaceTempView('result_sdf')
             result_sdf = result_sdf[result_sdf.from_node != result_sdf.to_node]  
             result_sdf.createOrReplaceTempView('result_sdf')
             result_sdf = result_sdf.filter("to_node is not null").cache()
             visit_sdf.createOrReplaceTempView('visit_sdf')
             frontier2_sdf = spark.sql('SELECT node as vnode from frontier_sdf')
             frontier2_sdf.createOrReplaceTempView('frontier2_sdf')
             visit_sdf = visit_sdf.unionAll(frontier2_sdf).cache()
             visit_sdf = visit_sdf.distinct()
             finaldepths_sdf.createOrReplaceTempView('finaldepths_sdf')
             finaldepths_sdf = finaldepths_sdf.unionAll(frontier_sdf).cache()
             finaldepths_sdf = finaldepths_sdf.distinct()
             frontier_sdf = spark.sql('SELECT to_node as node from result_sdf').cache()
             frontier_sdf.createOrReplaceTempView('frontier_sdf')
             frontier_sdf.repartition(100, frontier_sdf.node)
             visit_sdf.repartition(100, visit_sdf.vnode)
                            
        return finaldepths_sdf

In [22]:
comments_questions_sdf.createOrReplaceTempView('comments_questions_sdf')
nodes_sdf = spark.sql('SELECT * FROM comments_questions_sdf g '\
                           'WHERE g.from_node < 8')

In [23]:
nodes_sdf.show()

+---------+-------+-------------------+
|from_node|to_node|          timestamp|
+---------+-------+-------------------+
|        5|   4640|2008-09-17 05:31:33|
|        5|   4640|2008-09-17 06:14:24|
|        1|  14981|2008-09-17 11:43:58|
|        1|   7952|2008-09-18 17:07:19|
|        1|  17389|2008-09-19 11:39:26|
|        1|   1853|2008-09-20 20:06:42|
|        1|  20946|2008-09-30 02:06:42|
|        3|     26|2008-10-06 16:08:59|
|        1|   1768|2008-10-06 16:26:32|
|        1|  25210|2008-10-09 02:20:01|
|        5|      5|2008-10-09 18:34:34|
|        3|   6932|2008-10-09 22:05:31|
|        1|  17451|2008-10-16 14:27:05|
|        1|  10157|2008-10-17 21:05:53|
|        3|  28919|2008-10-18 05:27:52|
|        1|  23458|2008-10-18 15:32:28|
|        1|  17877|2008-10-19 16:41:57|
|        3|  23072|2008-11-01 06:45:58|
|        3|  11173|2008-11-01 07:00:48|
|        3|  19856|2008-11-01 07:04:49|
+---------+-------+-------------------+
only showing top 20 rows



In [60]:
reachable_sdf = transitive_closure(comments_questions_sdf,nodes_sdf,3)

successful2
successful 3
successful 4
successful 5
successful 6
successful2
successful 3
successful 4
successful 5
successful 6
successful2
successful 3
successful 4
successful 5
successful 6


In [61]:
reachable_sdf.show()

KeyboardInterrupt: 

In [13]:
origin_sdf = gt_sdf[gt_sdf['from_node'] < 3]

In [14]:
origin_sdf.show()

+---------+-------+
|from_node|to_node|
+---------+-------+
|        1|      2|
|        1|      3|
|        1|      4|
|        1|      5|
|        1|      6|
|        1|      7|
|        2|      9|
|        2|     10|
|        1|      1|
+---------+-------+



In [17]:
reachable_sdf = transitive_closure(gt_sdf,origin_sdf,2)

In [18]:
reachable_sdf.show()

+----+
|node|
+----+
|  13|
|   6|
|   3|
|   5|
|   9|
|   4|
|   8|
|   7|
|  10|
|  11|
|  14|
+----+

